In [48]:
import platform
import os
import torch
import pandas as pd
import numpy as np
from numpy import int64
from PIL.Image import Image
from diffusers import EulerDiscreteScheduler
from diffusers import StableDiffusionPipeline
from verta import Client
from verta.dataset import Path
from verta.dataset.entities import Dataset
from verta.dataset.entities import DatasetVersion
from verta.deployment import DeployedModel
from verta.registry import VertaModelBase, verify_io, task_type
from verta.registry.entities import RegisteredModel, RegisteredModelVersion
from verta.environment import Python
from verta.registry import data_type
from verta.endpoint import Endpoint
from verta.utils import ModelAPI

In [49]:
os.environ['VERTA_EMAIL'] = 'cory@verta.ai'
os.environ['VERTA_DEV_KEY'] = '154a34ad-2cd2-4d5d-b87c-2b809e075faa'
os.environ['VERTA_HOST'] = 'cj.dev.verta.ai'
client: Client = Client()
project_name = "Stable Diffusion v2 Example"
endpoint_name = "Stable_Diffusion_v2"
dataset_name = "Stable Diffusion v2 prebuilt pipeline"
default_image_width = 512
default_image_height = 512
default_guidance_scale = 9
default_num_inference_steps = 25
default_num_images = 1
data_path = 'data/'
pipeline_path = 'data/pipeline'

got VERTA_HOST from environment
got VERTA_EMAIL from environment
got VERTA_DEV_KEY from environment
connection successfully established


In [3]:
# create the local data directory to store the prebuilt assets
os.makedirs(
    os.path.dirname(data_path),
    exist_ok=True,
)

In [4]:
print("configuring scheduler")
model_id = "stabilityai/stable-diffusion-2-1"
scheduler: EulerDiscreteScheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

print("configuring pipeline")
revision = 'fp16'
torch_dtype = torch.float16

processor = platform.processor()
# initialize the image pipeline using custom setting for M1/M2 mac
if processor == 'arm':
    # ARM-based Macs do not support the fp16 revision nor the float16 dtype when initializing the pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler
    )
else:
    # x86 based OSes can use the fp16 revision and float16 dtype
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler,
        torch_dtype=torch_dtype,
        revision=revision,
    )

configuring scheduler
configuring pipeline


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

configuring pipeline device


In [5]:
# Store the pretrained model to disk
print("storing pretrained pipeline to disk")
StableDiffusionPipeline.save_pretrained(pipe, save_directory=pipeline_path)

storing pretrained pipeline to disk


In [6]:
# create a dataset out of the pretrained model
print("creating a dataset for the pretrained pipeline")
dataset: Dataset = client.get_or_create_dataset(name=dataset_name)

creating a dataset for the pretrained pipeline
created new Dataset: Stable Diffusion v2 prebuilt pipeline in workspace: shared


In [7]:
content: Path = Path([pipeline_path], enable_mdb_versioning=True)
dataset_version: DatasetVersion = dataset.create_version(content)

created new Dataset Version: 1 for Stable Diffusion v2 prebuilt pipeline
uploading data/pipeline/model_index.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/tokenizer/vocab.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/tokenizer/special_tokens_map.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/unet/diffusion_pytorch_model.bin to ModelDB
uploading part 55
upload complete
uploading data/pipeline/vae/diffusion_pytorch_model.bin to ModelDB
uploading part 6
upload complete
uploading data/pipeline/tokenizer/merges.txt to ModelDB
uploading part 1
upload complete
uploading data/pipeline/text_encoder/config.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/unet/config.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/vae/config.json to ModelDB
uploading part 1
upload complete
uploading data/pipeline/scheduler/scheduler_config.json to ModelDB
uploading part 1
upload complet

In [50]:
# define a custom verta model that will use prebuilt pipeline in the dataset to run the prediction
class StableDiffusionV2Generator(VertaModelBase):
    def __init__(self, artifacts):
        local_dataset_version: DatasetVersion = client.get_dataset(name=dataset_name).get_latest_version()
        local_path = '.'
        print("initializing from dataset version {}, downloading content to path {}".format(local_dataset_version.version, local_path))
        local_dataset_version.get_content().download(download_to_path=local_path)
        print("download complete, instantiating pipeline")
        pipeline: StableDiffusionPipeline = StableDiffusionPipeline.from_pretrained(pipeline_path)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("configuring pipeline device {}".format(device))
        self.pipeline: StableDiffusionPipeline = pipeline.to(device)
        print("pipeline ready for predictions")

    @verify_io
    def predict(self, prompts):
        prompt = prompts[0]
        guidance_scale = default_guidance_scale
        num_inference_steps = default_num_inference_steps
        print("executing pipeline with prompt '{}'".format(prompt))
        images = self.pipeline(
            prompt,
            num_images_per_prompt=default_num_images,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            height=default_image_height,
            width=default_image_width,
        ).images
        print("prediction complete for prompt '{}'".format(prompt))
        image: Image = images[0]
        print("resulting image is {}".format(image.info))
        return [image, image.height, image.width, guidance_scale, num_inference_steps]

In [51]:
# create the registered model
print("configuring verta model")
registered_model: RegisteredModel = client.get_or_create_registered_model(name=project_name, desc="Stable Diffusion v2 text-to-image generator", data_type=data_type.Image(), task_type=task_type.Other())

configuring verta model
got existing RegisteredModel: Stable Diffusion v2 Example


/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_utils.py:1457: UserWarning: Registered Model with name Stable Diffusion v2 Example already exists; cannot set `desc`, `labels`, `public_within_org`, or `visibility`
  warnings.warn(


In [52]:
# build the model api
model_api: ModelAPI = ModelAPI(
    pd.DataFrame.from_records(
        [{"prompt": "the prompt"}]),
    pd.DataFrame.from_records([
        {"image_data": np.array([0], dtype=int64),
         "image_height": default_image_height,
         "image_width": default_image_width,
         "guidance_scale": default_guidance_scale,
         "num_inference_steps": default_num_inference_steps}]),
)

In [53]:
# create the model version
version = "v19"
model_version: RegisteredModelVersion = registered_model.create_standard_model(
    name=version,
    model_cls=StableDiffusionV2Generator,
    model_api = model_api,
    environment=Python(requirements=Python.read_pip_file("requirements.txt"))
)

# log the dataset version that contains the prebuilt pipeline
model_version.log_dataset_version(key=dataset_name, dataset_version=dataset_version)

/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_pip_requirements_utils.py:213: UserWarning: 'diffusers~=0.10.2' does not use '=='; for reproducibility in deployment, it will be replaced with an exact pin of the currently-installed version
  warnings.warn(msg)
/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_pip_requirements_utils.py:213: UserWarning: 'torch~=1.13.0' does not use '=='; for reproducibility in deployment, it will be replaced with an exact pin of the currently-installed version
  warnings.warn(msg)
/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_pip_requirements_utils.py:213: UserWarning: 'requests~=2.28.1' does not use '=='; for reproducibility in deployment, it will be replaced with an exact pin of the currently-installed version
  warnings.warn(msg)
/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_pip_r

created new ModelVersion: v19
uploading model to Registry
uploading part 1
upload complete
uploading model_api.json to Registry
uploading part 1
upload complete
uploading custom_modules to Registry
uploading part 1
upload complete


In [ ]:
endpoint: Endpoint = client.get_or_create_endpoint(endpoint_name)
print("updating endpoint to model version {}".format(model_version.id))
endpoint.update(model_version, wait=True)

got existing Endpoint: Stable_Diffusion_v2
updating endpoint to model version version: v19
stage: unassigned
lock level: open
url: https://cj.dev.verta.ai/shared/registry/5/versions/21
time created: 2022-12-15 09:27:59.233000
time updated: 2022-12-15 09:28:15.572000
description: 
labels: []
attributes: {}
input description: 
hide input label: False
output description: 
hide output label: False
id: 21
registered model id: 5
experiment run id: 
artifact keys: ['custom_modules', 'model', 'model_api.json']
dataset version keys: ['Stable Diffusion v2 prebuilt pipeline']
code version keys: []
waiting for update....................................................................

In [ ]:
# generate an image from a text prompt
deployed_model: DeployedModel = endpoint.get_deployed_model()
prediction = deployed_model.predict(["An artistic logo for an AI company named Verta"])